In [1]:
#!/usr/bin/env python3
import os
from pymongo import MongoClient

def get_mongo_client():
    """
    Retrieves MongoDB URI from the environment variable MONGO_URI.
    """
    uri = os.getenv("MONGO_URI")
    if not uri:
        raise RuntimeError("Please set the MONGO_URI environment variable.")
    return MongoClient(uri)

def find_eligible_content(max_auditors=5):
    """
    Connects to Tel_QA database and finds content IDs with fewer than
    max_auditors total (real audits + pending placeholders).
    """
    client = get_mongo_client()
    db = client["Tel_QA"]
    
    qa_col     = db["QA_pairs"]
    audit_col  = db["audit_logs"]
    assign_col = db["assignment_placeholders"]
    
    # All content IDs defined in QA_pairs
    content_ids = qa_col.distinct("content_id")
    
    eligible = []
    for cid in content_ids:
        real_count    = audit_col.count_documents({"content_id": cid})
        pending_count = assign_col.count_documents({"content_id": cid})
        total         = real_count + pending_count
        
        if total < max_auditors:
            eligible.append({
                "content_id": cid,
                "real_audits": real_count,
                "pending": pending_count,
                "total": total
            })
    
    return eligible

def main():
    eligible = find_eligible_content(max_auditors=5)
    if not eligible:
        print("✅ All content audited!")
    else:
        print("Content IDs still available for auditing (fewer than 5 assignments):")
        for item in eligible:
            print(f" - ID {item['content_id']}: {item['real_audits']} audits, "
                  f"{item['pending']} pending, total {item['total']}")

if __name__ == "__main__":
    main()


✅ All content audited!
